In [2]:

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib agg
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os

In [3]:
source = os.getcwd()
source

'/Users/juliawang/Desktop/Direct/Spicy-Raman/model_train_and_validate'

## Data Importing

In [4]:
train_dir = os.path.join(source, 'trainingdata_Pictures')
validation_dir = os.path.join(source, 'validationdata_Pictures')

## Data Information

In [5]:
train_fluorescent_dir = os.path.join(train_dir, 'fluorescent_Pictures')
train_nylon_dir = os.path.join(train_dir, 'nylon_Pictures')
train_polyamide_dir = os.path.join(train_dir, 'polyamide_Pictures')
train_polyethylene_dir = os.path.join(train_dir, 'polyethylene_Pictures')
train_others_dir = os.path.join(train_dir, 'others_Pictures')

validation_fluorescent_dir = os.path.join(validation_dir, 'fluorescent_Pictures')
validation_nylon_dir = os.path.join(validation_dir, 'nylon_Pictures')
validation_polyamide_dir = os.path.join(validation_dir, 'polyamide_Pictures')
validation_polyethylene_dir = os.path.join(validation_dir, 'polyethylene_Pictures')
validation_others_dir = os.path.join(validation_dir, 'others_Pictures')

num_fluorescent_tr = len(os.listdir(train_fluorescent_dir))
num_nylon_tr = len(os.listdir(train_nylon_dir))
num_polyamide_tr = len(os.listdir(train_polyamide_dir))
num_polyethylene_tr = len(os.listdir(train_polyethylene_dir))
num_others_tr = len(os.listdir(train_others_dir))

num_fluorescent_val = len(os.listdir(validation_fluorescent_dir))
num_nylon_val = len(os.listdir(validation_nylon_dir))
num_polyamide_val = len(os.listdir(validation_polyamide_dir))
num_polyethylene_val = len(os.listdir(validation_polyethylene_dir))
num_others_val = len(os.listdir(validation_others_dir))


#num_cats_val = len(os.listdir(validation_cats_dir))
#num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_fluorescent_tr + num_polyamide_tr + num_nylon_tr + num_polyethylene_tr + num_others_tr
total_val = num_fluorescent_val + num_polyamide_val + num_nylon_val + num_polyethylene_val + num_others_val

#validation_cats_dir = os.path.join(validation_dir, 'cats')
#validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print("Total training images:", total_train)
print("Total validation images:", total_val)

Total training images: 463
Total validation images: 122


## Data Training

In [6]:
batch_size = 50
epochs = 15
IMG_HEIGHT = 480
IMG_WIDTH = 640

In [7]:
train_image_generator = ImageDataGenerator(rescale=1./255) 
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [8]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 463 images belonging to 5 classes.


In [9]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 122 images belonging to 5 classes.


In [10]:
sample_training_images, _ = next(train_data_gen)

In [12]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [13]:
plotImages(sample_training_images[:5])

/Users/juliawang/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  


In [11]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 480, 640, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 240, 320, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 240, 320, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 120, 160, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 160, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 60, 80, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 307200)            0

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/15


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("Spicy_Raman_Saved_Model")
zipfile.ZipFile("Spicy_Raman_Saved_Model.zip", mode='w').write("Spicy_Raman_Saved_Model.h5", compress_type=zipfile.ZIP_DEFLATED)

In [ ]:
class_names = ['Polyethylene', 'Fluorescent', 'Polyamide', 'Nylon', 'Unknown']
train_images = train_images/255.0
test_images = test_images/255.0

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape = (28,28)),
    keras.layers.Dense(128, activation = "relu"),
    keras.layers.Dense(10, activation = "softmax")
    ])

model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])